# Config Sweep: Finding Optimal Specialization Parameters

Systematically test **34 training configurations** across specialization mechanisms:
- Diversity bonus & niche pressure (Groups A, B)
- Freeze-Evolve phase ratios & mutation boost (Groups D, E)
- Pure evolution baselines (Group F)
- Field ON/OFF baselines (Group G)

Each config runs **3 seeds** with variable iterations per training mode:
- GRADIENT: 5 iters (~1.3M agent-steps)
- FREEZE_EVOLVE: 15 iters (~3.9M agent-steps)
- EVOLVE: 10 iters (~2.6M agent-steps)

## Metrics Tracked
| Metric | Description |
|--------|-------------|
| Hidden food collected | Coordination task success |
| Weight divergence | Genetic differentiation |
| Final population | Population stability |
| Regular food collected | Basic foraging |

## Runtime
- ~3 hours on TPU v6e + High-RAM
- Resume-safe: re-run skips completed configs
- Results saved to Drive after every config

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
CHECKPOINT_BASE = '/content/drive/MyDrive/emergence-lab'
SWEEP_DIR = os.path.join(CHECKPOINT_BASE, 'config_sweep')
os.makedirs(SWEEP_DIR, exist_ok=True)

REPO_DIR = '/content/emergence-lab'
GITHUB_USERNAME = "imashishkh21"

if not os.path.exists(REPO_DIR):
    !git clone https://github.com/{GITHUB_USERNAME}/emergence-lab.git {REPO_DIR}
else:
    !cd {REPO_DIR} && git pull

os.chdir(REPO_DIR)
!pip install -e ".[dev,phase5]" -q
!pip install rliable -q

import jax
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
print(f"Device count: {jax.device_count()}")

In [ ]:
import copy
import gc
import json
import os
import pickle
import time
import traceback
from dataclasses import dataclass
from datetime import datetime, timedelta

import jax
import jax.numpy as jnp
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from src.configs import Config, TrainingMode
from src.training.parallel_train import ParallelTrainer, sample_actions_per_agent
from src.agents.network import ActorCritic
from src.agents.policy import get_deterministic_actions
from src.environment.env import reset, step
from src.environment.obs import get_observations, obs_dim
from src.analysis.specialization import compute_weight_divergence

# --- Constants ---
NUM_SEEDS = 3
SEED_IDS = [0, 1, 2]
NUM_ENVS = 32
NUM_STEPS = 128
MAX_AGENTS = 64
SPI = NUM_ENVS * NUM_STEPS * MAX_AGENTS  # 262,144 Steps Per Iteration

# Variable iterations per training mode
ITERS_GRADIENT = 5       # ~1.3M agent-steps, ~4 min/config
ITERS_FREEZE_EVOLVE = 15 # ~3.9M agent-steps, ~8.5 min/config
ITERS_EVOLVE = 10        # ~2.6M agent-steps, ~6 min/config

ITERS_MAP = {
    TrainingMode.GRADIENT: ITERS_GRADIENT,
    TrainingMode.FREEZE_EVOLVE: ITERS_FREEZE_EVOLVE,
    TrainingMode.EVOLVE: ITERS_EVOLVE,
}

print(f"Steps Per Iteration (SPI): {SPI:,}")
print(f"GRADIENT: {ITERS_GRADIENT} iters = {ITERS_GRADIENT * SPI:,} agent-steps")
print(f"FREEZE_EVOLVE: {ITERS_FREEZE_EVOLVE} iters = {ITERS_FREEZE_EVOLVE * SPI:,} agent-steps")
print(f"EVOLVE: {ITERS_EVOLVE} iters = {ITERS_EVOLVE * SPI:,} agent-steps")

# Dependency check
assert hasattr(TrainingMode, 'FREEZE_EVOLVE'), "TrainingMode.FREEZE_EVOLVE not found!"
assert callable(sample_actions_per_agent), "sample_actions_per_agent not importable!"
print("\nDependency check PASSED")

In [ ]:
@dataclass
class SweepConfig:
    """A named config for the sweep."""
    name: str
    group: str
    config: Config


def make_base_config() -> Config:
    """Proven 64-agent hidden food base config."""
    cfg = Config()
    # Grid and agents
    cfg.env.grid_size = 32
    cfg.env.num_agents = 16
    cfg.env.num_food = 40
    # Evolution
    cfg.evolution.enabled = True
    cfg.evolution.max_agents = MAX_AGENTS
    cfg.evolution.starting_energy = 200
    cfg.evolution.food_energy = 100
    cfg.evolution.reproduce_threshold = 120
    cfg.evolution.reproduce_cost = 40
    cfg.evolution.mutation_std = 0.01
    # Hidden food (value_multiplier defaults to 5.0)
    cfg.env.hidden_food.enabled = True
    cfg.env.hidden_food.num_hidden = 3
    cfg.env.hidden_food.required_agents = 3
    cfg.env.hidden_food.reveal_distance = 3
    # Field
    cfg.field.diffusion_rate = 0.1
    cfg.field.decay_rate = 0.05
    cfg.field.write_strength = 1.0
    # Training
    cfg.train.total_steps = 10_000_000
    cfg.train.num_envs = NUM_ENVS
    cfg.train.num_steps = NUM_STEPS
    cfg.log.wandb = False
    cfg.log.save_interval = 0
    return cfg


def build_sweep_configs() -> list[SweepConfig]:
    """Build all 34 sweep configurations."""
    configs = []

    # ---- Group A1: diversity_bonus sweep (5 configs) ----
    for db in [0.0, 0.05, 0.1, 0.2, 0.5]:
        cfg = make_base_config()
        cfg.train.training_mode = TrainingMode.GRADIENT
        cfg.specialization.diversity_bonus = db
        configs.append(SweepConfig(
            name=f"A1_db{db}", group="A1_diversity_bonus", config=cfg,
        ))

    # ---- Group A3: mutation_std sweep (5 configs) ----
    for ms in [0.005, 0.01, 0.02, 0.05, 0.1]:
        cfg = make_base_config()
        cfg.train.training_mode = TrainingMode.GRADIENT
        cfg.evolution.mutation_std = ms
        configs.append(SweepConfig(
            name=f"A3_ms{ms}", group="A3_mutation_std", config=cfg,
        ))

    # ---- Group B: diversity x niche grid (6 configs) ----
    for db in [0.1, 0.2, 0.5]:
        for np_ in [0.05, 0.1]:
            cfg = make_base_config()
            cfg.train.training_mode = TrainingMode.GRADIENT
            cfg.specialization.diversity_bonus = db
            cfg.specialization.niche_pressure = np_
            configs.append(SweepConfig(
                name=f"B_db{db}_np{np_}", group="B_div_x_niche", config=cfg,
            ))

    # ---- Group G1: baseline field ON (1 config) ----
    cfg = make_base_config()
    cfg.train.training_mode = TrainingMode.GRADIENT
    configs.append(SweepConfig(
        name="G1_baseline_field_on", group="G_baseline", config=cfg,
    ))

    # ---- Group G2: baseline field OFF (1 config) ----
    cfg = make_base_config()
    cfg.train.training_mode = TrainingMode.GRADIENT
    cfg.field.decay_rate = 1.0
    cfg.field.diffusion_rate = 0.0
    cfg.field.write_strength = 0.0
    configs.append(SweepConfig(
        name="G2_baseline_field_off", group="G_baseline", config=cfg,
    ))

    # ---- Group D1: FE ratio sweep (5 configs) ----
    fe_ratios = [
        ("95_5",  14, 1),   # ~14:1 ratio
        ("90_10",  9, 1),   # ~9:1 ratio
        ("80_20",  4, 1),   # 4:1 ratio
        ("60_40",  3, 2),   # 3:2 ratio
        ("50_50",  1, 1),   # 1:1 ratio
    ]
    for label, grad_iters, evolve_iters in fe_ratios:
        cfg = make_base_config()
        cfg.train.training_mode = TrainingMode.FREEZE_EVOLVE
        cfg.freeze_evolve.gradient_steps = grad_iters * SPI
        cfg.freeze_evolve.evolve_steps = evolve_iters * SPI
        cfg.freeze_evolve.evolve_mutation_boost = 5.0
        configs.append(SweepConfig(
            name=f"D1_fe_ratio_{label}", group="D1_FE_ratio", config=cfg,
        ))

    # ---- Group D2: FE mutation boost sweep (4 configs) ----
    for boost in [2.0, 5.0, 10.0, 20.0]:
        cfg = make_base_config()
        cfg.train.training_mode = TrainingMode.FREEZE_EVOLVE
        cfg.freeze_evolve.gradient_steps = 4 * SPI   # 4:1 ratio
        cfg.freeze_evolve.evolve_steps = 1 * SPI
        cfg.freeze_evolve.evolve_mutation_boost = boost
        configs.append(SweepConfig(
            name=f"D2_fe_boost{int(boost)}", group="D2_FE_boost", config=cfg,
        ))

    # ---- Group E: FE + diversity rewards (4 configs) ----
    fe_div_combos = [
        ("4_1_db02_np01", 4, 1, 0.2, 0.1),
        ("3_2_db02_np01", 3, 2, 0.2, 0.1),
        ("4_1_db01_np005", 4, 1, 0.1, 0.05),
        ("3_2_db01_np005", 3, 2, 0.1, 0.05),
    ]
    for label, grad_i, evolve_i, db, np_ in fe_div_combos:
        cfg = make_base_config()
        cfg.train.training_mode = TrainingMode.FREEZE_EVOLVE
        cfg.freeze_evolve.gradient_steps = grad_i * SPI
        cfg.freeze_evolve.evolve_steps = evolve_i * SPI
        cfg.freeze_evolve.evolve_mutation_boost = 5.0
        cfg.specialization.diversity_bonus = db
        cfg.specialization.niche_pressure = np_
        configs.append(SweepConfig(
            name=f"E_fe_div_{label}", group="E_FE_diversity", config=cfg,
        ))

    # ---- Group F: pure evolve (3 configs) ----
    for ms in [0.01, 0.05, 0.1]:
        cfg = make_base_config()
        cfg.train.training_mode = TrainingMode.EVOLVE
        cfg.evolution.mutation_std = ms
        cfg.freeze_evolve.evolve_mutation_boost = 5.0
        configs.append(SweepConfig(
            name=f"F_evolve_ms{ms}", group="F_pure_evolve", config=cfg,
        ))

    return configs


# Build and verify
all_configs = build_sweep_configs()
print(f"Total configs: {len(all_configs)}")
assert len(all_configs) == 34, f"Expected 34 configs, got {len(all_configs)}"

# Sort: GRADIENT first, then FREEZE_EVOLVE, then EVOLVE
mode_order = {TrainingMode.GRADIENT: 0, TrainingMode.FREEZE_EVOLVE: 1, TrainingMode.EVOLVE: 2}
all_configs.sort(key=lambda sc: mode_order[sc.config.train.training_mode])

# Print summary
from collections import Counter
group_counts = Counter(sc.group for sc in all_configs)
mode_counts = Counter(sc.config.train.training_mode.value for sc in all_configs)
print(f"\nBy group: {dict(group_counts)}")
print(f"By mode: {dict(mode_counts)}")

# Time estimate
t_grad = mode_counts.get('gradient', 0) * 4
t_fe = mode_counts.get('freeze_evolve', 0) * 8.5
t_evolve = mode_counts.get('evolve', 0) * 6
print(f"\nEstimated runtime: {t_grad:.0f} + {t_fe:.0f} + {t_evolve:.0f} = {t_grad + t_fe + t_evolve:.0f} min (~{(t_grad + t_fe + t_evolve) / 60:.1f} hours)")

# Verify FREEZE_EVOLVE configs have steps as multiples of SPI
# (EVOLVE mode doesn't use freeze_evolve phase switching, so skip those)
for sc in all_configs:
    if sc.config.train.training_mode == TrainingMode.FREEZE_EVOLVE:
        gs = sc.config.freeze_evolve.gradient_steps
        es = sc.config.freeze_evolve.evolve_steps
        assert gs % SPI == 0, f"{sc.name}: gradient_steps {gs} not multiple of SPI"
        assert es % SPI == 0, f"{sc.name}: evolve_steps {es} not multiple of SPI"

print("\nAll FREEZE_EVOLVE configs verified: gradient_steps and evolve_steps are multiples of SPI")

# List all configs
print(f"\n{'Name':<30} {'Group':<20} {'Mode':<15} {'Iters':>5}")
print("-" * 75)
for sc in all_configs:
    iters = ITERS_MAP[sc.config.train.training_mode]
    print(f"{sc.name:<30} {sc.group:<20} {sc.config.train.training_mode.value:<15} {iters:>5}")

In [ ]:
# =============================================================================
# Eval Functions: shared-params AND per-agent
# =============================================================================

def run_hidden_food_eval_shared(network, params, config, key, num_eval_steps=200):
    """Evaluate using SHARED params (GRADIENT mode)."""
    env_state = reset(key, config)

    total_reward = 0.0
    regular_food = 0.0
    hf_collected = 0.0

    for _ in range(num_eval_steps):
        obs = get_observations(env_state, config)  # (max_agents, obs_dim)
        # get_deterministic_actions expects (num_envs, num_agents, obs_dim)
        actions = get_deterministic_actions(network, params, obs[None, :, :])
        actions = actions[0]  # (max_agents,)

        env_state, rewards, dones, info = step(env_state, actions, config)
        # Mask rewards by alive agents to avoid dead-slot noise
        alive_rewards = rewards * env_state.agent_alive.astype(jnp.float32)
        total_reward += float(jnp.sum(alive_rewards))
        regular_food += float(info['food_collected_this_step'])
        hf_collected += float(info['hidden_food_collected_this_step'])

    final_population = int(jnp.sum(env_state.agent_alive))
    return {
        'total_reward': total_reward,
        'regular_food_collected': regular_food,
        'hidden_food_collected': hf_collected,
        'final_population': final_population,
    }


def run_hidden_food_eval_per_agent(network, per_agent_params, config, key, num_eval_steps=200):
    """Evaluate using PER-AGENT params (EVOLVE/FE mode).

    per_agent_params: leaves shape (max_agents, ...) from a single env.
    """
    env_state = reset(key, config)

    # vmap network.apply with in_axes=(0, 0) for per-agent params
    batched_apply = jax.vmap(network.apply, in_axes=(0, 0))

    total_reward = 0.0
    regular_food = 0.0
    hf_collected = 0.0

    for _ in range(num_eval_steps):
        obs = get_observations(env_state, config)  # (max_agents, obs_dim)
        out = batched_apply(per_agent_params, obs)  # per_agent: (A, ...), obs: (A, D)
        logits = out[0]  # (max_agents, num_actions)
        actions = jnp.argmax(logits, axis=-1)  # (max_agents,) deterministic

        env_state, rewards, dones, info = step(env_state, actions, config)
        # Mask rewards by alive agents to avoid dead-slot noise
        alive_rewards = rewards * env_state.agent_alive.astype(jnp.float32)
        total_reward += float(jnp.sum(alive_rewards))
        regular_food += float(info['food_collected_this_step'])
        hf_collected += float(info['hidden_food_collected_this_step'])

    final_population = int(jnp.sum(env_state.agent_alive))
    return {
        'total_reward': total_reward,
        'regular_food_collected': regular_food,
        'hidden_food_collected': hf_collected,
        'final_population': final_population,
    }


def extract_eval_metrics(trainer, config):
    """Extract eval metrics from a trained ParallelTrainer.

    Returns list of dicts, one per seed.
    """
    ps = trainer._parallel_state
    if ps is None:
        return [{'error': 'no state'}] * trainer.num_seeds

    network = ActorCritic(
        hidden_dims=tuple(config.agent.hidden_dims),
        num_actions=6,
    )

    results = []
    for i in range(trainer.num_seeds):
        try:
            # Extract per-agent params from env 0 for this seed
            seed_agent_params_env0 = jax.tree.map(
                lambda x: x[i, 0], ps.env_state.agent_params
            )  # leaves: (max_agents, ...)

            alive_mask = np.array(ps.env_state.agent_alive[i, 0])  # (max_agents,)

            # Weight divergence
            div_result = compute_weight_divergence(seed_agent_params_env0, alive_mask)

            # Eval
            eval_key = jax.random.PRNGKey(42 + i)
            if config.train.training_mode == TrainingMode.GRADIENT:
                seed_params = jax.tree.map(lambda x: x[i], ps.params)
                eval_result = run_hidden_food_eval_shared(
                    network, seed_params, config, eval_key
                )
            else:
                eval_result = run_hidden_food_eval_per_agent(
                    network, seed_agent_params_env0, config, eval_key
                )

            results.append({
                'mean_divergence': float(div_result['mean_divergence']),
                'max_divergence': float(div_result['max_divergence']),
                'alive_count': int(np.sum(alive_mask)),
                **eval_result,
            })
        except Exception as e:
            results.append({'error': str(e)})

    return results

print("Eval functions defined.")

In [ ]:
# =============================================================================
# SMOKE TEST: 1 config, 1 iteration - verify full pipeline
# =============================================================================
print("SMOKE TEST: Running G1 baseline for 1 iteration...")
print("This verifies: train -> eval -> metrics before the full sweep.\n")

smoke_cfg = make_base_config()
smoke_cfg.train.training_mode = TrainingMode.GRADIENT

trainer = ParallelTrainer(
    config=smoke_cfg, num_seeds=1, seed_ids=[99],
    checkpoint_dir='/tmp/sweep_smoke', master_seed=0,
)
t0 = time.time()
metrics = trainer.train(
    num_iterations=1, checkpoint_interval_minutes=999,
    resume=False, print_interval=1,
)
print(f"\nTraining done in {time.time() - t0:.1f}s")

# Test eval extraction
eval_results = extract_eval_metrics(trainer, smoke_cfg)
print(f"Eval result: {eval_results[0]}")

# Verify
assert eval_results[0].get('mean_divergence') is not None, "Divergence missing!"
assert eval_results[0].get('hidden_food_collected') is not None, "HF collected missing!"
assert eval_results[0].get('final_population', 0) > 0, "Population crashed!"

del trainer
gc.collect()
jax.clear_caches()

print("\nSMOKE TEST PASSED - pipeline verified.")

In [ ]:
# =============================================================================
# MAIN SWEEP LOOP
# =============================================================================

# Load existing results if resuming
results_path = os.path.join(SWEEP_DIR, 'sweep_results.pkl')
if os.path.exists(results_path):
    with open(results_path, 'rb') as f:
        results = pickle.load(f)
    print(f"Loaded {len(results)} existing results")
else:
    results = {}


def save_results(results_dict):
    """Save results dict to Drive."""
    tmp_path = results_path + '.tmp'
    with open(tmp_path, 'wb') as f:
        pickle.dump(results_dict, f, protocol=pickle.HIGHEST_PROTOCOL)
    os.replace(tmp_path, results_path)


sweep_start = time.time()
total_configs = len(all_configs)
completed = sum(1 for r in results.values() if r.get('success'))

print(f"Starting sweep: {total_configs} configs, {completed} already complete")
print("=" * 80)

for idx, sc in enumerate(all_configs):
    # Skip if already done
    if sc.name in results and results[sc.name].get('success'):
        print(f"[{idx+1}/{total_configs}] {sc.name} - SKIPPED (already complete)")
        continue

    iters = ITERS_MAP[sc.config.train.training_mode]
    mode = sc.config.train.training_mode.value
    print(f"\n[{idx+1}/{total_configs}] {sc.name} ({sc.group}) | {mode} | {iters} iters")

    try:
        t0 = time.time()
        trainer = ParallelTrainer(
            config=sc.config, num_seeds=NUM_SEEDS, seed_ids=SEED_IDS,
            checkpoint_dir=f'/tmp/sweep_{sc.name}', master_seed=42 + idx,
        )
        train_metrics = trainer.train(
            num_iterations=iters,
            checkpoint_interval_minutes=999,
            resume=False, print_interval=5,
        )

        # Extract eval metrics
        eval_metrics = extract_eval_metrics(trainer, sc.config)
        train_time = time.time() - t0

        # Aggregate across seeds
        valid_evals = [e for e in eval_metrics if 'error' not in e]
        if valid_evals:
            agg = {}
            for key in valid_evals[0]:
                vals = [e[key] for e in valid_evals]
                agg[f"{key}_mean"] = float(np.mean(vals))
                agg[f"{key}_std"] = float(np.std(vals))
        else:
            agg = {}

        results[sc.name] = {
            'success': True,
            'group': sc.group,
            'mode': mode,
            'iters': iters,
            'train_time': train_time,
            'train_metrics': train_metrics,
            'eval_per_seed': eval_metrics,
            'eval_agg': agg,
        }

        hf_val = agg.get('hidden_food_collected_mean', 0.0)
        div_val = agg.get('mean_divergence_mean', 0.0)
        pop_val = agg.get('final_population_mean', 0.0)
        print(f"  Done in {train_time:.1f}s | "
              f"HF: {hf_val:.1f} | Div: {div_val:.4f} | Pop: {pop_val:.0f}")

    except Exception as e:
        print(f"  FAILED: {e}")
        results[sc.name] = {
            'success': False,
            'group': sc.group,
            'mode': mode,
            'error': traceback.format_exc(),
        }

    finally:
        try:
            del trainer
        except NameError:
            pass
        gc.collect()
        jax.clear_caches()

    # Save after every config
    save_results(results)

    # Progress
    completed = sum(1 for r in results.values() if r.get('success'))
    elapsed = time.time() - sweep_start
    if completed > 0:
        avg_time = elapsed / completed
        remaining = avg_time * (total_configs - completed)
        eta = datetime.now() + timedelta(seconds=remaining)
        print(f"  Progress: {completed}/{total_configs} | "
              f"Elapsed: {elapsed/60:.0f}min | ETA: {eta.strftime('%H:%M')}")

total_elapsed = time.time() - sweep_start
print("\n" + "=" * 80)
print(f"SWEEP COMPLETE! {completed}/{total_configs} successful in {total_elapsed/60:.0f} min")
print("=" * 80)

In [ ]:
# =============================================================================
# LEADERBOARD
# =============================================================================

# Reload results (in case of cell re-run)
if os.path.exists(results_path):
    with open(results_path, 'rb') as f:
        results = pickle.load(f)

# Build leaderboard rows
rows = []
for name, r in results.items():
    if not r.get('success'):
        rows.append({
            'name': name, 'group': r.get('group', '?'),
            'hf_collected': 0.0, 'divergence': 0.0,
            'pop_stability': 0.0, 'failed': True,
        })
        continue

    agg = r.get('eval_agg', {})
    hf = agg.get('hidden_food_collected_mean', 0.0)
    div = agg.get('mean_divergence_mean', 0.0)
    pop = agg.get('final_population_mean', 0.0)
    pop_stability = pop / MAX_AGENTS  # 0-1 scale

    rows.append({
        'name': name, 'group': r.get('group', '?'),
        'hf_collected': hf, 'divergence': div,
        'pop_stability': pop_stability, 'final_pop': pop,
        'failed': False, 'crashed': pop == 0,
    })

# Min-max normalization across successful, non-crashed configs
valid = [r for r in rows if not r['failed'] and not r.get('crashed', False)]

if valid:
    hf_vals = [r['hf_collected'] for r in valid]
    div_vals = [r['divergence'] for r in valid]

    hf_min, hf_max = min(hf_vals), max(hf_vals)
    div_min, div_max = min(div_vals), max(div_vals)

    def norm(v, vmin, vmax):
        return (v - vmin) / (vmax - vmin) if vmax > vmin else 0.5

    for r in rows:
        if r['failed'] or r.get('crashed', False):
            r['score'] = 0.0
        else:
            hf_norm = norm(r['hf_collected'], hf_min, hf_max)
            div_norm = norm(r['divergence'], div_min, div_max)
            r['score'] = 0.5 * hf_norm + 0.25 * div_norm + 0.25 * r['pop_stability']

    # Sort by score descending
    rows.sort(key=lambda r: r['score'], reverse=True)

    # Print top 20
    print(f"{'Rank':<5} {'Name':<30} {'Group':<20} {'HF_Col':>8} {'Diverg':>8} {'Pop':>6} {'Score':>7}")
    print("-" * 90)
    for i, r in enumerate(rows[:20]):
        status = ""
        if r['failed']:
            status = " FAILED"
        elif r.get('crashed', False):
            status = " CRASHED"

        print(f"{i+1:<5} {r['name']:<30} {r['group']:<20} "
              f"{r['hf_collected']:>8.1f} {r['divergence']:>8.4f} "
              f"{r.get('final_pop', 0):>6.0f} {r['score']:>7.3f}{status}")

    n_failed = sum(1 for r in rows if r['failed'])
    n_crashed = sum(1 for r in rows if r.get('crashed', False) and not r['failed'])
    print(f"\nTotal: {len(rows)} configs | {len(valid)} valid | {n_failed} failed | {n_crashed} crashed")
else:
    print("No valid results to rank.")

In [ ]:
# =============================================================================
# ANALYSIS PLOTS (4 figures)
# =============================================================================

# Only plot valid configs
plot_rows = [r for r in rows if not r['failed'] and not r.get('crashed', False)]

if not plot_rows:
    print("No valid data to plot.")
else:
    # Group colors
    unique_groups = sorted(set(r['group'] for r in plot_rows))
    try:
        cmap = plt.colormaps['tab10'].resampled(max(len(unique_groups), 1))
    except AttributeError:
        cmap = plt.cm.get_cmap('tab10', max(len(unique_groups), 1))
    group_colors = {g: cmap(i) for i, g in enumerate(unique_groups)}

    # --- Figure 1: Scatter - Divergence vs HF Collected ---
    fig1, ax1 = plt.subplots(figsize=(10, 7))
    for g in unique_groups:
        g_rows = [r for r in plot_rows if r['group'] == g]
        ax1.scatter(
            [r['divergence'] for r in g_rows],
            [r['hf_collected'] for r in g_rows],
            c=[group_colors[g]], label=g, s=80, alpha=0.8, edgecolors='black', linewidths=0.5,
        )
    # Annotate top 5
    top5 = sorted(plot_rows, key=lambda r: r['score'], reverse=True)[:5]
    for r in top5:
        ax1.annotate(r['name'], (r['divergence'], r['hf_collected']),
                     fontsize=7, ha='left', va='bottom',
                     xytext=(5, 5), textcoords='offset points')
    ax1.set_xlabel('Weight Divergence (mean cosine distance)')
    ax1.set_ylabel('Hidden Food Collected')
    ax1.set_title('Weight Divergence vs Hidden Food Collected')
    ax1.legend(fontsize=8, bbox_to_anchor=(1.02, 1), loc='upper left')
    fig1.tight_layout()
    fig1.savefig(os.path.join(SWEEP_DIR, 'scatter_div_vs_hf.png'), dpi=150)
    plt.show()

    # --- Figure 2: Top 10 bars (3 panels) ---
    top10 = sorted(plot_rows, key=lambda r: r['score'], reverse=True)[:10]
    names = [r['name'] for r in top10]

    fig2, axes = plt.subplots(1, 3, figsize=(18, 6))

    axes[0].barh(names[::-1], [r['hf_collected'] for r in top10][::-1], color='steelblue')
    axes[0].set_title('Hidden Food Collected')
    axes[0].set_xlabel('Count')

    axes[1].barh(names[::-1], [r['divergence'] for r in top10][::-1], color='coral')
    axes[1].set_title('Weight Divergence')
    axes[1].set_xlabel('Mean cosine distance')

    axes[2].barh(names[::-1], [r['score'] for r in top10][::-1], color='forestgreen')
    axes[2].set_title('Composite Score')
    axes[2].set_xlabel('Score')

    fig2.suptitle('Top 10 Configurations', fontsize=14)
    fig2.tight_layout()
    fig2.savefig(os.path.join(SWEEP_DIR, 'top10_bars.png'), dpi=150)
    plt.show()

    # --- Figure 3: Heatmap - Group B grid (div_bonus x niche_pressure) ---
    b_rows = [r for r in plot_rows if r['group'] == 'B_div_x_niche']
    if b_rows:
        db_vals = sorted(set(float(r['name'].split('_db')[1].split('_')[0]) for r in b_rows))
        np_vals = sorted(set(float(r['name'].split('_np')[1]) for r in b_rows))

        grid = np.zeros((len(np_vals), len(db_vals)))
        for r in b_rows:
            db = float(r['name'].split('_db')[1].split('_')[0])
            np_v = float(r['name'].split('_np')[1])
            i = np_vals.index(np_v)
            j = db_vals.index(db)
            grid[i, j] = r['hf_collected']

        fig3, ax3 = plt.subplots(figsize=(8, 5))
        im = ax3.imshow(grid, aspect='auto', cmap='YlOrRd')
        ax3.set_xticks(range(len(db_vals)))
        ax3.set_xticklabels([str(v) for v in db_vals])
        ax3.set_yticks(range(len(np_vals)))
        ax3.set_yticklabels([str(v) for v in np_vals])
        ax3.set_xlabel('diversity_bonus')
        ax3.set_ylabel('niche_pressure')
        ax3.set_title('Group B: Hidden Food Collected (div x niche grid)')
        for i in range(len(np_vals)):
            for j in range(len(db_vals)):
                ax3.text(j, i, f"{grid[i, j]:.0f}", ha='center', va='center',
                         fontsize=12, fontweight='bold')
        fig3.colorbar(im, ax=ax3)
        fig3.tight_layout()
        fig3.savefig(os.path.join(SWEEP_DIR, 'heatmap_div_x_niche.png'), dpi=150)
        plt.show()
    else:
        print("No Group B data for heatmap.")

    # --- Figure 4: Box plots per group ---
    fig4, ax4 = plt.subplots(figsize=(12, 6))
    group_data = {}
    for r in plot_rows:
        group_data.setdefault(r['group'], []).append(r['hf_collected'])

    groups_sorted = sorted(group_data.keys())
    data_for_box = [group_data[g] for g in groups_sorted]

    bp = ax4.boxplot(data_for_box, labels=groups_sorted, patch_artist=True)
    for patch, g in zip(bp['boxes'], groups_sorted):
        patch.set_facecolor(group_colors.get(g, 'lightblue'))

    ax4.set_ylabel('Hidden Food Collected')
    ax4.set_title('Hidden Food Collected by Group')
    plt.xticks(rotation=45, ha='right')
    fig4.tight_layout()
    fig4.savefig(os.path.join(SWEEP_DIR, 'boxplot_per_group.png'), dpi=150)
    plt.show()

    print(f"\nAll figures saved to {SWEEP_DIR}/")

In [ ]:
# =============================================================================
# SAVE RESULTS + RECOMMENDATIONS
# =============================================================================

# Save final results pickle (already saved incrementally, but save final version)
save_results(results)
print(f"Results saved to {results_path}")

# Save JSON summary (human-readable)
summary = {}
for name, r in results.items():
    if r.get('success'):
        summary[name] = {
            'group': r['group'],
            'mode': r['mode'],
            'train_time_s': r.get('train_time', 0),
            **r.get('eval_agg', {}),
        }
    else:
        summary[name] = {
            'group': r.get('group', '?'),
            'error': r.get('error', 'unknown')[:200],
        }

json_path = os.path.join(SWEEP_DIR, 'sweep_summary.json')
with open(json_path, 'w') as f:
    json.dump(summary, f, indent=2)
print(f"JSON summary saved to {json_path}")

# Print top 5 with full config values
print("\n" + "=" * 80)
print("TOP 5 CONFIGURATIONS")
print("=" * 80)

top5_names = [r['name'] for r in rows if not r['failed']][:5]
for rank, name in enumerate(top5_names, 1):
    r = results[name]
    agg = r.get('eval_agg', {})

    # Find the matching SweepConfig
    sc = next((s for s in all_configs if s.name == name), None)
    if sc is None:
        continue

    cfg = sc.config
    print(f"\n--- #{rank}: {name} (score: {rows[rank-1]['score']:.3f}) ---")
    print(f"  Group: {sc.group}")
    print(f"  Mode: {cfg.train.training_mode.value}")
    print(f"  HF collected: {agg.get('hidden_food_collected_mean', 0):.1f} +/- {agg.get('hidden_food_collected_std', 0):.1f}")
    print(f"  Divergence: {agg.get('mean_divergence_mean', 0):.4f}")
    print(f"  Population: {agg.get('final_population_mean', 0):.0f}")
    print(f"  Config params:")
    print(f"    diversity_bonus = {cfg.specialization.diversity_bonus}")
    print(f"    niche_pressure = {cfg.specialization.niche_pressure}")
    print(f"    mutation_std = {cfg.evolution.mutation_std}")
    if cfg.train.training_mode in (TrainingMode.FREEZE_EVOLVE, TrainingMode.EVOLVE):
        print(f"    gradient_steps = {cfg.freeze_evolve.gradient_steps}")
        print(f"    evolve_steps = {cfg.freeze_evolve.evolve_steps}")
        print(f"    evolve_mutation_boost = {cfg.freeze_evolve.evolve_mutation_boost}")
    print(f"    field: diffusion={cfg.field.diffusion_rate}, decay={cfg.field.decay_rate}, write={cfg.field.write_strength}")

print("\n" + "=" * 80)
print("Copy-paste ready config for specialization training notebook:")
print("=" * 80)

if top5_names:
    best = next(s for s in all_configs if s.name == top5_names[0])
    bc = best.config
    print(f"""\n# Best config from sweep: {best.name}
cfg.train.training_mode = TrainingMode.{bc.train.training_mode.name}
cfg.specialization.diversity_bonus = {bc.specialization.diversity_bonus}
cfg.specialization.niche_pressure = {bc.specialization.niche_pressure}
cfg.evolution.mutation_std = {bc.evolution.mutation_std}""")
    if bc.train.training_mode in (TrainingMode.FREEZE_EVOLVE, TrainingMode.EVOLVE):
        print(f"cfg.freeze_evolve.gradient_steps = {bc.freeze_evolve.gradient_steps}")
        print(f"cfg.freeze_evolve.evolve_steps = {bc.freeze_evolve.evolve_steps}")
        print(f"cfg.freeze_evolve.evolve_mutation_boost = {bc.freeze_evolve.evolve_mutation_boost}")

print(f"\nAll results saved to {SWEEP_DIR}/")